In [8]:
%reload_ext autoreload
%autoreload 2

import os
import pprint
import io

import pymongo
import json
import pandas as pd

from IPython.display import display, HTML

import climatools.lblnew.setup_bestfit as setup_bestfit
import climatools.lblnew.runrecord_bestfit as runrecord
import climatools.lblnew.pipeline as pipeline

In [2]:
client = pymongo.MongoClient('localhost', 27017, connect=False)

In [3]:
db = client.lblnew

In [4]:
collection = db.bestfit_lw

In [5]:
def make_doc(param=None):
    '''
    Makes a Mongodb document for an lblnew-bestfit run.
    
    Parameters
    ----------
    param: dict
        lblnew-bestfit input parameters.
    doc: dict
        lblnew-bestfit input (param) and output (flux and cooling rate)
    '''
    dir_fortran = pipeline.get_dir_case(param=param, 
                                        setup=setup_bestfit)
    
    fnames = [n for n in os.listdir(dir_fortran) if n in 
              ('output_wfluxg.dat', 'output_wcoolrg.dat',
               'output_fluxg.dat', 'output_coolrg.dat',
               'output_flux.dat', 'output_coolr.dat',
               'abscom.dat', 'dgdgs.dat',
               'kg_lin.dat', 'kg_nonlin.dat')]    
    
    fpaths = (os.path.join(dir_fortran, n) for n in fnames)
    s_outputs = (open(p, mode='r', encoding='utf-8').read() 
                 for p in fpaths)
    
    doc = {fname.split('.')[0]: s 
           for fname, s in zip(fnames, s_outputs)}
    doc['param'] = param
    
    return doc



def make_query(param=None):
    '''
    Returns the MongoDB query for a lblnew-bestfit 
    run's document.
    
    Parameters
    ----------
    param: dict
    lblnew-bestfit input parameters.
    '''
    return {'param.' + name: value for name, value in param.items()}
    

# Saving lblnew-bestfit runs in general

In [9]:
params = runrecord.params()

In [72]:
docs = (make_doc(param=param) for param in params)
collection.insert_many(docs)

In [73]:
#collection.delete_many({'param.atmpro': 'trp'})

In [74]:
collection.count()

144

# Saving best-fit runs

These runs are lblnew-bestift runs, and they are also considered the "best-fit" to the line-by-line calculations and are to be imported into clirad-lw.

In [16]:
import climatools.lblnew.bestfit_params as bfit
import climatools.lblnew.export as export

In [29]:
def get_mbs():
    '''
    Returns all (molecule, band) pairs implemented in clirad-lw.
    '''
    fs = [export.h2o_gasbands, export.co2_gasbands,
          export.o3_gasbands, export.n2o_gasbands,
          export.ch4_gasbands]
    return [mb for f in fs for mb in f()]

In [31]:
params = [bfit.kdist_params(molecule=m, band=b) 
          for m, b in get_mbs()]

In [23]:
docs = (make_doc(param=p) for p in params)
for doc in docs:
    print(doc.keys())

dict_keys(['kg_nonlin', 'kg_lin', 'output_wcoolrg', 'output_wfluxg', 'output_flux', 'output_fluxg', 'output_coolr', 'output_coolrg', 'abscom', 'dgdgs', 'param'])
dict_keys(['kg_nonlin', 'kg_lin', 'output_wcoolrg', 'output_wfluxg', 'output_flux', 'output_fluxg', 'output_coolr', 'output_coolrg', 'abscom', 'dgdgs', 'param'])


In [32]:
for p in params:
    print(p['molecule'], p['band'])
    rs = collection.find(make_query(param=p))
    if rs:
        for r in rs:
            print(r.keys())
            
    print()

    
        

h2o 1
dict_keys(['_id', 'param', 'kg_nonlin', 'kg_lin', 'debug_dtku', 'output_wcoolrg', 'output_wfluxg', 'output_flux', 'output_fluxg', 'output_coolr', 'output_coolrg', 'abscom', 'dgdgs', 'pt_ktable'])

h2o 2
dict_keys(['_id', 'param', 'kg_nonlin', 'kg_lin', 'debug_dtku', 'output_wcoolrg', 'output_wfluxg', 'output_flux', 'output_fluxg', 'output_coolr', 'output_coolrg', 'abscom', 'dgdgs', 'pt_ktable'])

h2o 3a
dict_keys(['_id', 'param', 'kg_nonlin', 'kg_lin', 'debug_dtku', 'output_wcoolrg', 'output_wfluxg', 'output_flux', 'output_fluxg', 'output_coolr', 'output_coolrg', 'abscom', 'dgdgs', 'pt_ktable'])

h2o 3b
dict_keys(['_id', 'param', 'kg_nonlin', 'kg_lin', 'debug_dtku', 'output_wcoolrg', 'output_wfluxg', 'output_flux', 'output_fluxg', 'output_coolr', 'output_coolrg', 'abscom', 'dgdgs', 'pt_ktable'])

h2o 3c
dict_keys(['_id', 'param', 'kg_nonlin', 'kg_lin', 'debug_dtku', 'output_wcoolrg', 'output_wfluxg', 'output_flux', 'output_fluxg', 'output_coolr', 'output_coolrg', 'abscom', 'dgdgs